In [1]:
from HANEmotionAnalyze import *
import torch
import torch.nn as nn
import numpy
import time, math
import torch.utils.data as data
import os
import pandas as pd
import json

In [2]:
# import torch.functional as F
# 
embedding = nn.Embedding(10, 3)
input = torch.LongTensor([[0, 1,2,4,5],[1, 4,3,2,9]])
print(embedding(input))
embedding2 = nn.Embedding(10, 3, padding_idx=2)
# input = torch.LongTensor([[1,2,0,5, 6,7,8,9]])
print(embedding2(input))
weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding3 = nn.Embedding.from_pretrained(weight)
input = torch.LongTensor([1])
print(embedding3(input))
input = torch.LongTensor([0])
print(embedding3(input))

tensor([[[-5.1958e-01, -1.3948e-01, -9.0974e-05],
         [ 1.8373e-03,  1.2931e+00, -5.4605e-01],
         [-1.6162e-01,  4.1798e-01, -3.7759e-01],
         [-4.6572e-01,  6.1565e-01, -2.9826e-01],
         [ 3.0229e-01,  1.0494e+00,  4.9972e-01]],

        [[ 1.8373e-03,  1.2931e+00, -5.4605e-01],
         [-4.6572e-01,  6.1565e-01, -2.9826e-01],
         [-1.4307e+00,  9.7822e-02, -5.6423e-01],
         [-1.6162e-01,  4.1798e-01, -3.7759e-01],
         [ 1.0096e+00,  7.0409e-01, -9.4448e-01]]],
       grad_fn=<EmbeddingBackward>)
tensor([[[-0.4019,  2.1143,  0.4684],
         [ 1.4231,  0.8628, -1.3925],
         [ 0.0000,  0.0000,  0.0000],
         [ 1.2632,  1.2053, -0.3622],
         [ 0.2257,  0.9233,  0.7332]],

        [[ 1.4231,  0.8628, -1.3925],
         [ 1.2632,  1.2053, -0.3622],
         [ 0.5502, -1.1008, -0.9046],
         [ 0.0000,  0.0000,  0.0000],
         [ 1.4089,  0.4509, -1.3207]]], grad_fn=<EmbeddingBackward>)
tensor([[4.0000, 5.1000, 6.3000]])
tensor([[1.0

In [3]:
num_embeddings = 111111
model = HAN(num_embeddings)
print(model)

HAN(
  (embed): Embedding(5845, 200, padding_idx=0)
  (GRU1): GRU(200, 50, batch_first=True, bidirectional=True)
  (self_attention1): SelfAttention(
    (W): Linear(in_features=100, out_features=100, bias=True)
    (U): Linear(in_features=100, out_features=1, bias=True)
  )
  (GRU2): GRU(50, 50, batch_first=True, bidirectional=True)
  (self_attention2): SelfAttention(
    (W): Linear(in_features=100, out_features=100, bias=True)
    (U): Linear(in_features=100, out_features=1, bias=True)
  )
  (fc): Linear(in_features=100, out_features=10, bias=True)
)


## 读取词向量
建立词语列表

In [ ]:
from gensim.models import KeyedVectors

file = 'Tencent_AILab_ChineseEmbedding/Tencent_AILab_ChineseEmbedding.txt'
wv_from_text = KeyedVectors.load_word2vec_format(file, binary=False)
wv_from_text.init_sims(replace=True)  # 神奇，很省内存，可以运算most_similar

In [9]:
def compute_ngrams(word, num_min, num_max):
    ngrams =[]
    for ngram_length in range(num_min, min(len(word), num_max) + 1):
        for i in range(len(word) - ngram_length + 1):
            # print(i, i + ngram_length)
            ngrams.append(word[i : i + ngram_length])
    # print(ngrams)
    return list(set(ngrams))

print(compute_ngrams('you',  num_min = 1, num_max = 3))
print(compute_ngrams('I Think',  num_min = 1, num_max = 3))
print(compute_ngrams('中华人民共和国万岁',  num_min = 1, num_max = 3))

['you', 'y', 'yo', 'o', 'ou', 'u']
['h', 'hin', ' ', 'i', 'Th', 'nk', 'in', 'I', 'T', 'I T', ' Th', 'k', 'I ', 'hi', 'Thi', 'n', ' T', 'ink']
['和', '万', '国万', '中华人', '万岁', '人民共', '民共', '民', '岁', '人民', '人', '共', '国', '华', '和国', '国万岁', '共和国', '共和', '中华', '中', '华人', '华人民', '和国万', '民共和']


In [ ]:
# 从词向量文本文件 wv_from_text 中获取词向量，如果获取到直接返回，若没有获取到，那么把这个词拆开
# 成为 ngrams 的新词组，并在 wv_from_text 中找新词组中的词向量并相加取平均，最后得到平均词向量输出
def wordVec(word, wv_from_text, min_n = 1, max_n = 3):
    '''
    ngrams_single/ngrams_more,主要是为了当出现oov的情况下,最好先不考虑单字词向量
    '''
    # 确认词向量维度
    word_size = wv_from_text.wv.syn0[0].shape[0]

    # 如果在词典之中，直接返回词向量
    if word in wv_from_text.wv.vocab.keys():
        return wv_from_text[word]
    else:
        # 计算word的ngrams词组
        ngrams = compute_ngrams(word, min_n = min_n, max_n = max_n)
        # 不在词典的情况下
        word_vec = numpy.zeros(word_size, dtype=numpy.float32)
        ngrams_found = 0
        ngrams_single = [ng for ng in ngrams if len(ng) == 1]
        ngrams_more = [ng for ng in ngrams if len(ng) > 1]
        # 先只接受2个单词长度以上的词向量
        for ngram in ngrams_more:
            if ngram in wv_from_text.wv.vocab.keys():
                word_vec += wv_from_text[ngram]
                ngrams_found += 1
                #print(ngram)
        # 如果，没有匹配到，那么最后是考虑单个词向量
        if ngrams_found == 0:
            for ngram in ngrams_single:
                word_vec += wv_from_text[ngram]
                ngrams_found += 1
        if word_vec.any():
            return word_vec / max(1, ngrams_found)
        else:
            raise KeyError('all ngrams for word %s absent from model' % word)

In [ ]:
# def isNan(a):
#     return a != a

# class RatingData4Bert(data.Dataset):
#     def __init__(self, path, tokenizer:Tokenizer, max_row = -1, trainTestRate = 0.85, isTrain = True):
#         self.token_list = []
#         self.label_list = []
#         self.token_positions = torch.tensor([i for i in range(100)])
#
#         print('地址不应该包含 ratings.csv   path:',path) # 地址不应该包含 ratings.csv
#
#         ratings_clean_filename = os.path.join(path, 'ratings_clean.csv')
#         ratings_filename = os.path.join(path, 'ratings.csv')
#         if os.path.isfile(ratings_clean_filename):
#             clean_pd = pd.read_csv(ratings_clean_filename)
#         else:
#             print('没有找到缓存的文件%s, 读取源文件%s'%(ratings_clean_filename, ratings_filename))
#             ratings_pd = pd.read_csv(ratings_filename)
#             print('开始生成缓存文件%s'%(ratings_clean_filename))
#             clean_pd = pd.DataFrame({
#                 'userId':[],
#                 'restId':[],
#                 'rating':[],
#                 'comment':[],
#             })
#             nonRatingCount = 0
#             for i, row in ratings_pd.iterrows():
#                 if max_row != -1 and i > max_row:
#                     break
#                 if not isinstance(row['comment'], str) or row['comment'] == '':
#                     # print(i + 1, row['comment'])
#                     nonRatingCount += 1
#                     continue
#                 r0 = row['rating']
#                 r1 = row['rating_env']
#                 r2 = row['rating_flavor']
#                 r3 = row['rating_service']
#                 if r0 == '' or isNan(r0): r0 = 0 # 假设总评分为 0 表示未评分
#                 if r1 == '' or isNan(r1): r1 = 3
#                 if r2 == '' or isNan(r2): r2 = 3
#                 if r3 == '' or isNan(r3): r3 = 3
#                 r0 = round(r0 * 0.5 + (r1 + r2 + r3) * 0.1666666)
#                 if i % 10000 == 9999:
#                     print(i + 1, r0)
#
#                 token = tokenizer.encode(text=str(row['comment']), max_length=100, pad_to_max_length = True)
#                 # print('token', token)
#                 # token = [101] + token + [102]
#
#                 newRow = DotDict()
#                 newRow.userId = [row['userId']]
#                 newRow.restId = [row['restId']]
#                 newRow.rating = [r0]
#                 newRow.comment = [json.dumps(token)]
#
#                 clean_pd = clean_pd.append(pd.DataFrame(newRow), ignore_index=True)
#             print('空的评论数量： %d'%(nonRatingCount))
#             clean_pd.to_csv(ratings_clean_filename)
#
#         # 读取
#         if isTrain:
#             temp_pd = clean_pd[ : int(len(clean_pd) * trainTestRate)]
#         else:
#             temp_pd = clean_pd[int(len(clean_pd) * trainTestRate) : ]
#
#         for i, row in temp_pd.iterrows():
#             if max_row != -1 and i > max_row:
#                 break
#
#             self.label_list.append(torch.tensor(row['rating']).long())
#             self.token_list.append(torch.from_numpy(numpy.array( json.loads(row['comment']) ) ).long())
#
#     def __getitem__(self, index):
#         # print(self.token_list[index], self.label_list[index], self.token_positions)
#         return self.token_list[index], self.label_list[index], self.token_positions
#
#     def __len__(self):
#         return len(self.label_list)
    